# Graph Setup

The following notebook will create 2 data sets. One data set will include the edge list for our social network graph. The other data set will include node attributes.

In [1]:
#Import libraries
from pyspark.sql.functions import explode, col, size
import warnings

warnings.filterwarnings("ignore")

In [2]:
#Run necessary notebooks
%run Data-Retrieval.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 21:54:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
friend_count = user.select("user_id", size("friends").alias("friend_count"))

In [4]:
friend_count.take(1).show()

24/03/28 21:54:31 WARN BlockManager: Block rdd_4_5 could not be removed as it was not found on disk or in memory
24/03/28 21:54:33 WARN MemoryStore: Not enough space to cache rdd_4_10 in memory! (computed 67.1 MiB so far)
24/03/28 21:54:34 WARN BlockManager: Persisting block rdd_4_10 to disk instead.
24/03/28 21:54:34 ERROR Executor: Exception in task 5.0 in stage 0.0 (TID 5)
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.sql.execution.columnar.BasicColumnBuilder.build(ColumnBuilder.scala:81)
	at org.apache.spark.sql.execution.columnar.ComplexColumnBuilder.org$apache$spark$sql$execution$columnar$NullableColumnBuilder$$super$build(ColumnBuilder.scala:93)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.build(NullableColumnBuilder.scala:67)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.build$(NullableColumnBuilder.scala:66

24/03/28 21:54:37 ERROR TaskSetManager: Task 5 in stage 0.0 failed 1 times; aborting job
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_7 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_11 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_0 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_4 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_3 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_2 failed due to exception org.apache.spark.TaskKilledException.
24/03/28 21:54:38 WARN BlockManager: Block rdd_4_3 could not be removed as it was not found on disk or in memory
24/03/28 21:54:38 WARN BlockManager: Putting block rdd_4_8 failed due t

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/quagliam/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/quagliam/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/quagliam/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/quagliam/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Jav

ConnectionRefusedError: [Errno 111] Connection refused